In [1]:
import torch
from torch import optim
from tqdm import tqdm

import numpy as np
import copy
import json

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
USE_CUDA = True
use_device = torch.device('cuda' if USE_CUDA else 'cpu')

In [2]:
from data_util import Schema

root = '/notebooks/sinica/dataset/'

schema_root = root+'schema_mr5.txt'
relation_data = root+'skincare_mr5.train'
dev_data = root+'skincare_mr5.dev'
test_data = root+'skincare_mr5.test'

word_to_ix = '/notebooks/sinica/code/word_to_ix.pkl'


schema = Schema(schema_root)

In [3]:
# ========hyper-parameter-set==========
MAX_LEN = 100
BATCH_SIZE = 32

EMBEDDING_DIM = 64
HIDDEN_DIM1 = 30  #30
HIDDEN_DIM2 =  16 #16
LABEL_EMBED_DIM = 8 #8

ATTN_OUT = 8 #8

n_iter = 20

In [4]:
from data_util import BIOLoader

loader = BIOLoader(relation_data, MAX_LEN, BATCH_SIZE, schema, 
                   word_to_ix=word_to_ix, shuffle=True, device=use_device)
dev_loader = BIOLoader(dev_data, MAX_LEN, BATCH_SIZE, schema, 
                       word_to_ix=loader.word_to_ix, device=use_device)
test_loader = BIOLoader(test_data, MAX_LEN, BATCH_SIZE, schema, 
                        word_to_ix=loader.word_to_ix, device=use_device)

In [5]:
from models_v1_5 import JointERE

total_exps = 10
scores = []
class_scores = []

param_list = [loader.vocab_size, EMBEDDING_DIM, HIDDEN_DIM1, HIDDEN_DIM2, LABEL_EMBED_DIM,
             ATTN_OUT, schema]


for i in tqdm(range(total_exps)):
    model = JointERE(*param_list).to(use_device)
    model.fit(loader, dev_loader, n_iter=n_iter)
    
    e_score, er_score, all_er_score = model.score(test_loader, silent=True, rel_detail=True, analyze=True)
    
    scores.append((e_score, er_score))
    class_scores.append(all_er_score)
    
    del model

  5%|▌         | 1/20 [00:26<08:14, 26.04s/it]

epoch: 1 | ent loss 0.7656 | rel loss 1.0699 | total loss 1.8356
         | val ent loss 0.6965 | val rel loss 0.5067



 10%|█         | 2/20 [00:52<07:51, 26.17s/it]

epoch: 2 | ent loss 0.3801 | rel loss 0.3726 | total loss 0.7527
         | val ent loss 0.4871 | val rel loss 0.2114



 15%|█▌        | 3/20 [01:18<07:25, 26.20s/it]

...

epoch: 19 | ent loss 0.0670 | rel loss 0.0866 | total loss 0.1536
          | val ent loss 0.0612 | val rel loss 0.0051



100%|██████████| 20/20 [09:15<00:00, 27.80s/it]


epoch: 20 | ent loss 0.0395 | rel loss 0.0682 | total loss 0.1077
          | val ent loss 0.0481 | val rel loss 0.0078

Entity detection score
precision  	 recall  	 fbeta_score
0.909 		 0.923 		 0.916 	
Entity+Relation detection score 
precision  	 recall  	 fbeta_score  	
0.825 		 0.464 		 0.594 	
confusion matrix 
TP  	 fp  	 tn  	 fn
170 	 36 	 0 	 196 	

Relation error count: 23


100%|██████████| 10/10 [1:37:09<00:00, 582.94s/it]

Relation type 0
precision  	 recall  	 fbeta_score  	
0.667 		 0.500 		 0.571 	

Relation type 1
precision  	 recall  	 fbeta_score  	
0.744 		 0.533 		 0.621 	

Relation type 2
precision  	 recall  	 fbeta_score  	
0.941 		 0.640 		 0.762 	

Relation type 3
precision  	 recall  	 fbeta_score  	
0.778 		 0.528 		 0.629 	

Relation type 4
precision  	 recall  	 fbeta_score  	
0.833 		 0.114 		 0.200 	



In [18]:
from scipy.stats import describe
e_p = describe([s[0][0] for s in scores]) 
e_r = describe([s[0][1] for s in scores])
e_f1 = describe([s[0][2] for s in scores])
print("E Statistics")
print("Precision: \t mean {:.3f} w/std {:.3f}".format(e_p.mean, e_p.variance**0.5))
print("Recall: \t mean {:.3f} w/std {:.3f}".format(e_r.mean, e_r.variance**0.5))
print("F1 score: \t mean {:.3f} w/std {:.3f}".format(e_f1.mean, e_f1.variance**0.5))

E Statistics
Precision: 	 mean 0.908 w/std 0.006
Recall: 	 mean 0.927 w/std 0.011
F1 score: 	 mean 0.918 w/std 0.006


In [19]:
er_p = describe([s[1][0] for s in scores]) 
er_r = describe([s[1][1] for s in scores])
er_f1 = describe([s[1][2] for s in scores])
print("E+R Statistics")
print("Precision: \t mean {:.3f} w/std {:.3f}".format(er_p.mean, er_p.variance**0.5))
print("Recall: \t mean {:.3f} w/std {:.3f}".format(er_r.mean, er_r.variance**0.5))
print("F1 score: \t mean {:.3f} w/std {:.3f}".format(er_f1.mean, er_f1.variance**0.5))

E+R Statistics
Precision: 	 mean 0.777 w/std 0.037
Recall: 	 mean 0.564 w/std 0.074
F1 score: 	 mean 0.649 w/std 0.038


In [20]:
for i, rel in enumerate(schema['relation']):
    er_p = describe([s[0][i] for s in class_scores]) 
    er_r = describe([s[1][i] for s in class_scores])
    er_f1 = describe([s[2][i] for s in class_scores])
    print("==============================================")
    print("E+R Statistics ({})".format(schema.rid2tag[i]))
    print("Precision: \t mean {:.3f} w/std {:.3f}".format(er_p.mean, er_p.variance**0.5))
    print("Recall: \t mean {:.3f} w/std {:.3f}".format(er_r.mean, er_r.variance**0.5))
    print("F1 score: \t mean {:.3f} w/std {:.3f}".format(er_f1.mean, er_f1.variance**0.5))


E+R Statistics (ApplyTo)
Precision: 	 mean 0.613 w/std 0.070
Recall: 	 mean 0.529 w/std 0.088
F1 score: 	 mean 0.561 w/std 0.056
E+R Statistics (ApplyIn)
Precision: 	 mean 0.713 w/std 0.049
Recall: 	 mean 0.645 w/std 0.098
F1 score: 	 mean 0.671 w/std 0.041
E+R Statistics (Feel)
Precision: 	 mean 0.877 w/std 0.041
Recall: 	 mean 0.718 w/std 0.062
F1 score: 	 mean 0.787 w/std 0.033
E+R Statistics (ApprovedOf)
Precision: 	 mean 0.772 w/std 0.050
Recall: 	 mean 0.628 w/std 0.058
F1 score: 	 mean 0.691 w/std 0.043
E+R Statistics (Relieve)
Precision: 	 mean 0.790 w/std 0.104
Recall: 	 mean 0.266 w/std 0.168
F1 score: 	 mean 0.365 w/std 0.178
